# 00 Gerating artificial signal peptides

## 0 Introduction:
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda:
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

In [2]:
import numpy as np
import pandas as pd
import random
import os

In [3]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str


In [4]:
def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
    
    
    return sequence        

In [5]:
def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


In [6]:
def split_peptides_sequences(df_signalPP:pd.DataFrame): 
    '''Split each AA for each position'''
    peptides_split = []
    for k,v in df_signalPP.iterrows(): 
        sequence = []
        for seq in v['sequence']: 
            sequence.append(seq)
        peptides_split.append(sequence)
    
    # make a dataframe
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides

## 1 Amino acid probability matrix

Lets import our df_pwn dataframe that was made in a previous notebook and use the position specific scoring matrix for the generation of novel signal peptides

In [7]:
from google.colab import drive 
drive.mount('/content/home')

Mounted at /content/home


In [8]:
df_pwn =  pd.read_csv('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/02_All_signal_peptides/df_pwn_68_positions.csv')
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.069877,0.001889,0.002833,0.001889,0.032106,0.021719,0.064212,0.037771,0.208687,0.084986,...,0.022663,0.028329,0.058546,0.193579,0.033994,0.018886,0.052880,0.014164,0.028329,0.000000
2,0.044381,0.007554,0.006610,0.007554,0.152975,0.047214,0.019830,0.050047,0.018886,0.190746,...,0.020774,0.052880,0.020774,0.053824,0.131256,0.053824,0.060434,0.025496,0.022663,0.000000
3,0.057602,0.001889,0.004721,0.004721,0.089707,0.019830,0.016053,0.064212,0.058546,0.175637,...,0.033994,0.041549,0.043437,0.056657,0.163362,0.100094,0.021719,0.010387,0.026440,0.000000
4,0.072710,0.004721,0.004721,0.007554,0.054769,0.032106,0.019830,0.077432,0.053824,0.155807,...,0.041549,0.040604,0.036827,0.045326,0.138810,0.096317,0.043437,0.023607,0.032106,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001889,0.000944,0.000000,0.000000,0.000000,0.997167
59,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.998111
60,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.998111
61,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [9]:
amino_acids = list(df_pwn.columns.values)

In [10]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())

## 2 Load best model from AutoML

In [11]:

!pip install h2o==3.40.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 MB 7.6 MB/s eta 0:00:00


In [12]:
import h2o
from h2o.automl import H2OAutoML

In [13]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp95zoexrt
  JVM stdout: /tmp/tmp95zoexrt/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp95zoexrt/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_unknownUser_e2vsel
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [14]:
h2o.__version__

'3.40.0.1'

In [15]:
best_model = h2o.load_model("/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/06_H2O_AutoML/06.1_Best_models/First_round/m30_DeepLearning_grid_1_AutoML_2_20230222_74710_model_5.model")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20230222_74710_model_5


Status of Neuron Layers: predicting MM_N_peptide_abundance, regression, gaussian distribution, Quadratic loss, 61,701 weights/biases, 732.3 KB, 304,013 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  --------------------  ----------  ---------------------  -------------------  --------------------  -----------------------
    1        615      Input             10.0
    2        100      RectifierDropout  40.0       0.0   0.0   0.15402221043693182   0.23625540733337402   0.0         -0.038140358922893976  0.10787653923034668  0.2525724750588628    0.25475525856018066
    3        1        Linear                       0.0   0.0   0.009403525316320155  0.020257048308849335  0.0         0.10333934525846417    0.10956910252571106  -0.08605526456670021  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0006201367116055927
RMSE: 0.024902544279763718
MAE: 0.0016178878632761011
RMSLE: 0.017479906422663657
Mean Residual Deviance: 0.0006201367116055927

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.001383662580170757
RMSE: 0.03719761524843706
MAE: 0.0048922401668397095
RMSLE: 0.028117549919822144
Mean Residual Deviance: 0.001383662580170757

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00488458  0.00322001  0.0130098     0.00557146    0.00548844    0.00485233    0.0058972     0.0041314     0.00296465    0.00318942    0.00213285    0.00160823
mean_residual_deviance  0.00137632  0.00278783  0.00916567    0.00164904    0.00111819    0.000719876   0.000477532   0.000374579   0.000138844   2.24663e-05   6.34567e-05   3.35485e-05
mse                     0.00137632  0.00278783  0.00916567    0.00164904    0.00111819    0.000719876   0.000477532   0.000374579   0.000138844   2.24663e-05   6.34567e-05   3.35485e-05
r2                      -0.168901   0.463878    0.00958452    -0.00817074   -0.0178689    -0.000134252  -0.0536192    -0.0216354    0.00192077    -1.47644      -0.185211     0.0625643
residual_deviance       0.00137632  0.00278783  0.00916567    0.00164904    0.00111819    0.000719876   0.000477532   0.000374579   0.000138844   2.24663e-05   6.34567e-05   3.35485e-05
rmse                    0.0268104   0.0270293   0.0957375     0.0406084     0.0334394     0.0268305     0.0218525     0.019354      0.0117832     0.00473987    0.00796597    0.00579211
rmsle                   0.0219567   0.0184126   0.0661309     0.0339551     0.028919      0.0237013     0.0198986     0.0177024     0.0111582     0.00472062    0.00774493    0.00563566

Scoring History: 
    timestamp            duration           training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  -----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-02-22 08:09:48  0.000 sec                            0         0             0          nan              nan                  nan             nan
    2023-02-22 08:09:48  18 min 42.166 sec  10602 obs/sec     1.70877   1             1813       0.0373193        0.00139273           0.0140212       

## 3 Signal peptide predictor algorithm

In [16]:
def signal_peptide_predictor(list_of_probabilities, amino_acids, n_peptides,  number_of_iterations:int)-> pd.DataFrame:
    '''Predicts best signal peptides from a number of iterations'''

    data = pd.DataFrame()
    for i in range(0,number_of_iterations):
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len = 22 )
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        # make the df into categorical values
        for column in df_test.columns:
            if column != 'MM_N_peptide_abundance':
                df_test[column] = df_test[column].asfactor()

        #predict
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        if len(data) == 0: 
            data = new_TO_NATURE_peptides.copy()
        else: 
            # concat to precious predictions
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending = False)
            data = data[0:1000]   

        if i % 1000 == 0:
            print(f"Iteration {i}")
            
    return data

In [17]:
%%time
lets_predict_signal_peptides = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids,n_peptides = 1000,  
                                                        number_of_iterations = 2000)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
lets_predict_signal_peptides

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,predictions
134,M,R,V,A,A,A,S,N,L,L,...,L,V,L,A,P,A,N,N,P,0.152499
639,M,L,M,A,S,S,C,F,L,L,...,I,Y,H,L,P,A,V,-,-,0.148933
599,M,A,N,S,S,A,C,L,M,L,...,I,S,T,L,C,R,-,-,-,0.147671
87,M,L,N,V,S,L,V,L,T,L,...,G,A,T,A,P,A,A,-,-,0.147459
16,M,K,R,V,S,Y,R,F,F,A,...,L,V,T,L,A,L,A,Q,-,0.126004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,M,R,M,P,W,Y,H,L,F,G,...,L,S,V,A,A,F,V,-,-,0.050450
135,M,L,V,S,H,G,K,F,L,A,...,F,S,V,A,A,A,A,-,-,0.050439
452,M,L,T,R,S,C,F,L,F,L,...,L,V,A,A,A,-,-,-,-,0.050404
341,M,W,S,S,N,A,L,A,L,A,...,L,A,A,L,P,A,Q,-,-,0.050400


In [ ]:
lets_predict_signal_peptides.to_excel('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/09_Generate_artificial_SPs/SPL30_74710_GASPs_0314.xlsx', index=False)